In [1]:
import sys 
sys.path.append("../../../")
from simulation_classes.eta.PathPlanner import PathPlanner

In [2]:
# initial_params
initial_time =20
v_0=20
initial_params = {"time": initial_time, "speed":v_0}

# ideal_params
v_exit = 30
ideal_arrival_time = 80
ideal_params_at_end = {"ideal_arrive_time": ideal_arrival_time, "ideal_speed": v_exit}

# car_spec
max_acc = 3
max_dec = 2
v_max = 45
car_spec = {"max_acc":max_acc, "max_dec":max_dec, "v_max":v_max}

#その他の設計
L_max = 500

pathPlanner = PathPlanner(car_spec = car_spec, initial_params = initial_params, 
                          ideal_params_at_end=ideal_params_at_end, COURSE_LENGTH = L_max)

pathPlanner.solve_path(priority="speed")

{'is_possible': True, 'Reason': 'OK'}
PROFILE:  DCA

m1=6.67, m2=48.33, m3=5.00
D1=133.33, D2=483.33, D3=75.00

m1=8.33, m2=44.17, m3=7.50
D1=145.83, D2=220.83, D3=93.75

m1=7.50, m2=46.25, m3=6.25
D1=140.62, D2=346.88, D3=85.94

m1=7.92, m2=45.21, m3=6.88
D1=143.49, D2=282.55, D3=90.23

m1=8.12, m2=44.69, m3=7.19
D1=144.73, D2=251.37, D3=92.09

m1=8.02, m2=44.95, m3=7.03
D1=144.12, D2=266.88, D3=91.19

m1=8.07, m2=44.82, m3=7.11
D1=144.43, D2=259.10, D3=91.64

m1=8.05, m2=44.88, m3=7.07
D1=144.28, D2=262.99, D3=91.42

m1=8.03, m2=44.92, m3=7.05
D1=144.20, D2=264.93, D3=91.30
9
m1の解=8.04

m1=8.04, m2=44.90, m3=7.06
D1=144.24, D2=263.96, D3=91.36
距離:499.56
[{'ACC': -2, 'duration': 7.060546875, 'initial_speed': 20}, {'ACC': 0, 'duration': 44.899088541666664, 'initial_speed': 5.87890625}, {'ACC': 3, 'duration': 8.040364583333334, 'initial_speed': 5.87890625}]


In [151]:
"""
クラスを書き直したことでこれは不要になった
"""

def calc_controlled_eta(initial_params, ideal_params_at_end, car_spec, L_max ):
    v_0 = initial_params["speed"]
    v_lim = car_spec["v_max"]
    v_exit = ideal_params_at_end["ideal_speed"]
    a_max = car_spec["max_acc"]
    a_dec = car_spec["max_dec"]
    t_0 = initial_params["time"]
    t_end = ideal_params_at_end["ideal_arrive_time"]
    L_max = L_max

    if v_0 > v_lim or v_exit > v_lim:
        ValueError("Enter Speedが最高速度を上回っています")

    if v_0 > v_exit:
        ValueError("Enter Speedが出口速度を上回っています")

    if (t_end - t_0) * a_max + v_0 < v_exit:
        print("不可能: 全力で加速するのみ")
        return False

    if (t_end - t_0) * v_0 >= L_max: # Trueだった場合、早く着きすぎているということ
        print("Possible: 早く着きすぎた")
        return True
        
    # ここより上の議論は(v_0, v_exit) と (t_0, t_end) だけで完結する話

    
    # 以下、距離まで織り込んだ議論 
    # 終点で速度をv_exitにするという条件のもとで頑張る
    t_margin = t_end - t_0
    time_to_reach_v_exit = (v_exit - v_0) / a_max
    v_max_possible = (t_margin - time_to_reach_v_exit) / (a_max + a_dec) * a_dec * a_max + v_exit

    print("可能的最高速度:", v_max_possible)

    if v_max_possible > v_lim:
        # この場合は一定速度 v_lim で走る区間がある、s1+s2+s3 > LならOKのやつ
        print("a")
        s_1 = (v_lim**2 - v_0 ** 2) / 2 / a_max
        s_3 = (v_lim**2 - v_exit ** 2) / 2 / a_dec
        t_1 = (v_lim - v_0) / a_max
        t_3 = (v_lim - v_exit) / a_dec
        t_2 = t_end - t_0 - t_1 - t_3
        S = s_1 + s_3 + t_2 * v_lim

        print("可能的最高速度:", v_lim)
        print("t_2=",t_2)
        print("t_3=",t_3)
        print("カバー距離:", S)
        print("達成可能性:", (t_2 < 0 or S > L_max))

        return (t_2 < 0 or S > L_max)

    # これより下は v_lim <= v_maxの場合。要するに行けるとこまで加速する
    # v_0で入ってからv_exitで抜けるまでの総走行距離を計算する
    print("定速区間なし")
    
    S = (v_max**2 - v_0**2) / 2 / a_max + (v_max**2 - v_0**2) / 2 / a_dec
    total_time = (v_max - v_0)/a_max + (v_max - v_exit) / a_dec
    can_cover_the_distance = S > L_max
    is_in_time = total_time < t_margin
    print("カバー距離:", S)
    print(can_cover_the_distance, is_in_time)
    print("達成可能性:", ( is_in_time and can_cover_the_distance))

    return (is_in_time and can_cover_distance)
    